In [165]:
from datetime import datetime,timedelta
from ftplib import FTP
import pandas as pd
enddate=datetime.now()
days=3
startdate=enddate-timedelta(days=days)
dates=[]
date=startdate
while date<=enddate:
    if date.isoweekday()<6:
        dates.append(date.strftime('%Y%m%d'))
    date=date+timedelta(1)
print(dates)

ftp = FTP('106.12.3.198')     # connect to host, default port
ftp.login()                     # user anonymous, passwd anonymous@
ftp.cwd('new_tdx\T0002\export')               # change  directory
ftp.encoding='gbk'

['20200101', '20200102', '20200103']


In [166]:
filenamelist=[]
for date in dates:
    filename="板块指数"+date+".txt"
    try:
        with open(filename, 'wb') as fp:
            ftpcommand="RETR "+filename
            ftp.retrbinary(ftpcommand, fp.write)
    except Exception as e:
        print("读取文件:",filename,"发生错误：",e)
    else:
        filenamelist.append(filename)
ftp.quit()


读取文件: 板块指数20200101.txt 发生错误： 550 The system cannot find the file specified. 


'221 Goodbye.'

In [167]:
i=1
for file in filenamelist:
    with open(file,'r',encoding="gbk") as f:
        a=f.readlines()
        data=[]
        for j in a:
            data.append(j[:-1].split('\t')[:-1])
        locals()["df"+str(i)]=pd.DataFrame(data[1:-1],columns=data[0])
        #j[:-1].split('\t')[:-1],第一个[:-1]去掉\t\n,第二个[:-1]去掉最后一个空项目，剩下都是列名
        i=i+1
        
        


In [176]:
df2[['涨停数']] = df2[['涨停数']].astype(int) 

In [178]:
df2.sort_values(by="涨停数" , ascending=False)[:20]


,代码,名称,涨幅%,现价,涨跌,涨速%,量比,涨跌数,涨停数,总金额,...,均价,开盘%,现均差%,流通市值,AB股总市值,流通股(亿),总股本(亿),短期形态,中期形态,长期形态
71,880516,ST板块,0.85,873.34,7.34,0.01,0.98,"94,26",22,4309850112,...,878.76,0.18,-0.63,3773.60亿,4504.64亿,1202.06,1384.71,,,
47,880879,低价股,1.04,833.44,8.60,0.05,1.13,"116,43",18,14121149440,...,833.44,0.22,0.00,11331.09亿,13014.16亿,5109.30,5850.37,,,
30,880946,区块链,1.37,801.96,10.80,0.14,1.26,"147,45",14,66888081408,...,803.62,0.35,-0.21,18341.12亿,24873.58亿,1935.49,2338.91,,,
139,880880,近期强势,0.47,432.06,2.02,0.11,1.36,"41,51",12,55430197248,...,431.58,1.20,0.11,7157.28亿,12076.94亿,616.15,837.59,,,
91,880868,高贝塔值,0.69,975.03,6.69,0.04,1.23,"55,41",12,55986331648,...,980.79,0.10,-0.59,9145.24亿,17145.13亿,768.38,1198.50,,,
162,880207,北京板块,0.37,1322.96,4.88,0.11,1.12,"209,116",11,87663984640,...,1329.34,0.23,-0.48,121172.01亿,141418.28亿,18303.36,20672.54,,,
5,880494,互联网,2.36,2778.19,64.16,0.07,1.50,"54,15",11,29510113280,...,2778.96,1.20,-0.03,5040.23亿,8708.83亿,527.76,768.93,,,
142,880892,高质押股,0.46,650.58,2.97,0.03,1.11,"105,78",11,22695645184,...,651.98,0.21,-0.22,12142.13亿,16276.19亿,2222.75,2772.32,,,
57,880360,农林牧渔,0.96,1686.06,16.09,0.14,1.58,"49,19",11,18872571904,...,1713.26,0.63,-1.63,8069.85亿,10394.59亿,606.60,705.74,,,
20,880791,网红经济,1.57,1096.57,16.98,0.08,1.69,"27,19",10,21704837120,...,1100.50,1.96,-0.36,2961.34亿,6016.67亿,327.73,483.41,,,


In [186]:

HEADER = '''
    <html>
        <head>
            <meta charset="UTF-8">
        </head>
        <body>
    '''
FOOTER = '''
        </body>
    </html>
    '''
 
with open('a.html', 'w',encoding='utf-8') as f:
    f.write(HEADER)
    for df in [df1, df2]:
        #f.write('<h1><strong>' + df +'</strong></h1>')
        f.write(df.to_html(classes='dd'))
        f.write(FOOTER)